In [1]:
import pandas as pd
import numpy as np
import pickle

# Adding a tag for the topics in focus: "gay marriage" and "abortion"
def add_tag(row):
    title = row['topic'].lower().strip()
    if title.find('abortion') > -1 :
        row['tag'] = 'abortion'
    elif title.find('gay marriage') > -1 :
        row['tag'] = 'gay marriage'
    else:
        row['tag'] = 'NA'
    return row

data_cross_path = 'data/cross-topic/{}.csv'
data_within_path = 'data/within-topic/{}.csv'

cross_train_df = pd.read_csv(data_cross_path.format('train_rand'), index_col='id', escapechar='\\')
cross_dev_df = pd.read_csv(data_cross_path.format('dev_rand'), index_col='id', escapechar='\\')
cross_test_df =  pd.read_csv(data_cross_path.format('test_rand'),index_col='id', escapechar='\\')

within_train_df =  pd.read_csv(data_within_path.format('train_rand'), index_col='id', escapechar='\\')
within_dev_df =  pd.read_csv(data_within_path.format('dev_rand'), index_col='id', escapechar='\\')
within_test_df =  pd.read_csv(data_within_path.format('test_rand'), index_col='id', escapechar='\\')

# apply add_tag
cross_train_df = cross_train_df.apply(add_tag, axis=1)
cross_dev_df = cross_dev_df.apply(add_tag, axis=1)
cross_test_df = cross_test_df.apply(add_tag, axis=1)

within_train_df = within_train_df.apply(add_tag, axis=1)
within_dev_df = within_dev_df.apply(add_tag, axis=1)
within_test_df = within_test_df.apply(add_tag, axis=1)

In [11]:
all_wiki = []
with open("data/eng_news_2016_1M/eng_news_2016_1M-sentences.txt", encoding="UTF-8") as f:
    for line in f:
        items = line.split("\t")
        all_wiki.append(items[1])
print(len(all_wiki))
print(all_wiki[0])

1000000
$0$0$0$0$0Donor Frustration$0$0One cannot but imagine Donor frustration and their desire to withdraw funding to the Ministry of Health.



In [15]:
from pytorch_pretrained_bert.tokenization import BertTokenizer

# bert_model = "bert-large-uncased-whole-word-masking"
bert_model = "bert-large-uncased"
tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=True)

In [22]:
tags = ['gay marriage', 'abortion', 'all']

,argument1,argument1_id,argument2,argument2_id,debate_id,is_same_side,topic,tag
id,,,,,,,,
2607,accepted. pro may extend their arguments to th...,2a0d32eb-2019-04-18T11:46:44Z-00004-000,"i""m pro-life. just think about it, your murder...",2a0d32eb-2019-04-18T11:46:44Z-00004-000,2a0d32eb-2019-04-18T11:46:44Z,False,abortion (pro life),abortion
14632,ultrasounds fit well with pro-choice concepts.,475596d3-2019-04-17T11:47:21Z-00031-000,ultrasounds are a procedure any pregnant woman...,475596d3-2019-04-17T11:47:21Z-00031-000,475596d3-2019-04-17T11:47:21Z,True,mandatory ultrasounds before abortions,abortion
84168,marriage is defined as between a man and woman,d2f4b1cd-2019-04-17T11:47:27Z-00092-000,marriage is celebrated because of the assumpti...,d2f4b1cd-2019-04-17T11:47:27Z-00092-000,d2f4b1cd-2019-04-17T11:47:27Z,False,"gay marriage, debate on same sex marriage",gay marriage
58293,"there is no inviolable ""right to life"" in abor...",b67fc3fb-2019-04-17T11:47:41Z-00102-000,"parents must ""control their bodies"" or else ri...",b67fc3fb-2019-04-17T11:47:41Z-00102-000,b67fc3fb-2019-04-17T11:47:41Z,False,abortion,abortion
85569,"al rantell, a homosexual talk-show host in la....",d2f4b1cd-2019-04-17T11:47:27Z-00154-000,denying marriage to infertile would be too costly,d2f4b1cd-2019-04-17T11:47:27Z-00154-000,d2f4b1cd-2019-04-17T11:47:27Z,False,"gay marriage, debate on same sex marriage",gay marriage
51234,the reproductive rights movement has no genoci...,b67fc3fb-2019-04-17T11:47:41Z-00024-000,legal abortion protects women with serious ill...,b67fc3fb-2019-04-17T11:47:41Z-00024-000,b67fc3fb-2019-04-17T11:47:41Z,True,abortion,abortion
16228,a woman has the right to her own body. not nea...,47ca97e8-2019-04-18T18:54:04Z-00004-000,abortion should be illegal in the united states.,47ca97e8-2019-04-18T18:54:04Z-00004-000,47ca97e8-2019-04-18T18:54:04Z,False,abortion,abortion
41532,abortion is important to reducing over-populat...,b67fc3fb-2019-04-17T11:47:41Z-00012-000,"life is an individual right, not a privilege, ...",b67fc3fb-2019-04-17T11:47:41Z-00012-000,b67fc3fb-2019-04-17T11:47:41Z,True,abortion,abortion
89518,children have a claim to biological parents,d2f4b1cd-2019-04-17T11:47:27Z-00028-000,gay spouses can helpfully adopt orphaned kids....,d2f4b1cd-2019-04-17T11:47:27Z-00028-000,d2f4b1cd-2019-04-17T11:47:27Z,False,"gay marriage, debate on same sex marriage",gay marriage


In [27]:
for tag in tags:
    print(tag)
    all_tokens = []
    if tag != "all":
        selected_args = cross_train_df[cross_train_df.tag==tag]
    else:
        selected_args = cross_train_df
    all_args = list(selected_args.argument1) + list(selected_args.argument2)
    for arg in all_args:
        tokens = tokenizer.tokenize(arg)
        all_tokens.append(tokens)
    with open(tag + ".tsv", "w") as f:
        for arg in all_tokens:
            f.write(" ".join(arg) + "\n")

gay marriage
abortion
all


In [28]:
all_tokens = []
for s in all_wiki:
    tokens = tokenizer.tokenize(s)
    all_tokens.append(tokens)
with open("wiki.tsv", "w") as f:
    for s in all_tokens:
        f.write(" ".join(s) + "\n")